In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
import random as rd
import backtrader as bt # 导入 Backtrader
import backtrader.indicators as btind # 导入策略分析模块
import backtrader.feeds as btfeeds # 导入数据模块

## data feed 模块
Data Feed 在 Backtrader 中扮演一个“数据传递者”的角色，给策略有序的提供数据以及数据的索引位置

### 1. self.datas 
策略函数中的 **self.datas 是数据表格的集合（是一个 Data Feeds 数据馈送对象 )** ，对应通过 Cerebro 导入的行情数据表格的集合（可能只导入了一只证券的行情数据，也可能导入了 N 只证券的行情数据）。在这个集合中，数据表格是按照导入的顺序依次确定索引位置，第一个导入的数据表格的索引位置为 0 ，之后的依次递增，如下图所示：
<figure class="half">
    <img src=../pic/bt_datafeed_1.png width="650">   
</figure>
编写策略调用行情数据时，有多种方式：   

1. 2种正向索引方式：  
    a.带中括号的常规方式 → self.datas[X]；  
    b.不带中括号的缩写方式 → self.dataX， **(没有s)** 其中 X 对应索引位置编号 0,1,2,3,...,N；  
2. 使用负向索引位置编号 -1,-2,-3,...,-N 调用数据时，不支持缩写，必须带中括号[ ]，如 self.datas[-1]；
3. 默认情况下，Backtrader 指向的是 self.datas 中的第一个导入的数据集，所以该数据集的调用方式可以直接省略索引号，写法能最简洁，有 3 种等价形式：  
   **self.datas[0] ↔ self.data0 ↔ self.data**；  
4. 通过表格名称调用数据： **self.getdatabyname('stockN')** ，表格名称是在导入数据时通过 name 参数设置的。

In [9]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        # 打印数据集和数据集对应的名称
        print("-------------self.datas-------------")
        print(self.datas)
        print("-------------self.data-------------")
        print(self.data._name, self.data) # 返回第一个导入的数据表格，缩写形式
        print("-------------self.data0-------------")
        print(self.data0._name, self.data0) # 返回第一个导入的数据表格，缩写形式
        print("-------------self.datas[0]-------------")
        print(self.datas[0]._name, self.datas[0]) # 返回第一个导入的数据表格，常规形式
        print("-------------self.datas[1]-------------")
        print(self.datas[1]._name, self.datas[1]) # 返回第二个导入的数据表格，常规形式
        print("-------------self.datas[-1]-------------")
        print(self.datas[-1]._name, self.datas[-1]) # 返回最后一个导入的数据表格
        print("-------------self.datas[-2]-------------")
        print(self.datas[-2]._name, self.datas[-2]) # 返回倒数第二个导入的数据表格
        print("-------------self.getdatabyname-------------")
        print(self.datas[0]._name, self.getdatabyname(self.datas[0]._name)) # 指定名字返回倒数第一个导入的数据表格   
        
cerebro = bt.Cerebro()
st_date = datetime(2019,1,2)
ed_date = datetime(2021,1,28)

daily_price = pd.read_csv("../bt_data/daily_price.csv", parse_dates=['datetime'])
data1 = daily_price.query(f"sec_code=='600466.SH'").set_index('datetime').drop(columns=['sec_code'])
data2 = daily_price.query(f"sec_code=='603228.SH'").set_index('datetime').drop(columns=['sec_code'])

# 添加 600466.SH 的行情数据
datafeed1 = bt.feeds.PandasData(dataname=data1,
                                fromdate=st_date,
                                todate=ed_date)
cerebro.adddata(datafeed1, name='600466.SH')
# 添加 603228.SH 的行情数据
datafeed2 = bt.feeds.PandasData(dataname=data2,
                                fromdate=st_date,
                                todate=ed_date)
cerebro.adddata(datafeed2, name='603228.SH')
cerebro.addstrategy(TestStrategy)
rasult = cerebro.run()

-------------self.datas-------------
[<backtrader.feeds.pandafeed.PandasData object at 0x0000015997FDA9A0>, <backtrader.feeds.pandafeed.PandasData object at 0x0000015997C3A3D0>]
-------------self.data-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997FDA9A0>
-------------self.data0-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997FDA9A0>
-------------self.datas[0]-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997FDA9A0>
-------------self.datas[1]-------------
603228.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997C3A3D0>
-------------self.datas[-1]-------------
603228.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997C3A3D0>
-------------self.datas[-2]-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015997FDA9A0>
-------------self.getdatabyname-------------
600466.SH <backtrader.feeds.pandafeed.PandasData object at 0x0000015

### 2.base concept
#### **a. lines**  
Backtrader 将**数据表格的列**拆成了一个个 **line** 线对象，**一列→一个指标→该指标的时间序列→一条线 line**。  
Backtrader 默认情况下要求导入的数据表格要包含 7 个字段：'datetime'、 'open'、 'high'、 'low'、 'close'、 'volume'、 'openinterest' ，这 7 个字段序列就对应了 7 条 line 。  
其实给列赋予“线”的概念也很好理解，回测过程中用到的时间序列行情数据可视化后就是一条条曲线：close 曲线、 open 曲线、high 曲线 ......  

#### 调用line  
每一个 Data Feed 对象都有一个 lines 属性。可以将 **lines 属性看作是 line 的集合** ，所以想要调用具体的某一条线，就**通过 lines 属性**来调用：
<figure class="half">
    <img src=../pic/bt_datafeed_2.png width="450">   
</figure>

 1. 访问 lines 属性：**xxx.lines，可简写成 xxx.l**;  
 2. 访问 lines 属性 中具体某条线：xxx.lines.name，可简写成 xxx.lines_name，其中 name 对应线的名称;  
 3. 一般可以套用“先调用某张数据表格，再调用这张表格中具体的某根 line”的逻辑依次编写代码；   
 4. 可以通过 getlinealiases() 方法查看 Data Feed 对象包含哪些线。   



In [ ]:
# 访问第一个数据集的 close 线
self.data.lines.close # 可省略 lines 简写成：self.data.close
self.data.lines_close # 可省略 lines 简写成：self.data_close
# 访问第二个数据集的 open 线
self.data1.lines.close # 可省略 lines 简写成：self.data1.close
self.data1.lines_close # 可省略 lines 简写成：self.data1_close
# 注：只有从 self.datas 调用 line 时可以省略 lines，调用 indicators 中的 line 时不能省略

# 如果你能清楚的记住数据表格中每条线的位置，也可以通过索引位置（整数）来访问，同样支持简写形式：
# 完整形式：
self.datas[X].lines[Y]；
# 简写形式：
self.dataX.lines[Y]、self.dataX_Y；
# 说明：X 对应单个数据表格在数据表格集合中的索引位置，Y 对应某条线在数据表格中的索引位置 。


#### 提取line上的数据点
Data Feeds 序列中的元素是一张张数据表格、lines 序列中的元素是一条条 line、line 序列中的元素是一个个数据点  
Backtrader 创建了一套新的索引规则和一个切片方法 get()：

1. 索引规则：索引位置编号结合了时间信息，**0 号位置永远指向当前时间点的数据** ，-1 号位置指向前一个时间点的数据，然后依次回退 （backwards）-2、-3、-4、-5、......；1 号位置指向下一天的数据，然后依次向前（forwards）2、3、4、......；  

2. 切片方法：get(ago=0, size=1) 函数，其中 **ago 对应数据点的索引位置** ，即**从 ago 时间点开始往前取 size 个数据点** 。默认情况下是取当前最新时点（ago=0）的那一个数据（size=1）；

3. 在编写策略时，上面提到的对数据点的索引切片操作一般在 next() 函数中涉及较多，而 \_\_init__() 中涉及较少，因为\_\_init__() 中一般是对 一整条 line 进行操作（运算）。



In [13]:
# 实用举例
class TestStrategy1(bt.Strategy):
    def __init__(self):
        self.count = 0 # 用于计算 next 的循环次数
        # 打印数据集和数据集对应的名称
        print("------------- init 中的索引位置-------------")
        print("0 索引：",'datetime',self.data1.lines.datetime.date(0), 'close',self.data1.lines.close[0])
        print("-1 索引：",'datetime',self.data1.lines.datetime.date(-1),'close', self.data1.lines.close[-1])
        print("-2 索引",'datetime', self.data1.lines.datetime.date(-2),'close', self.data1.lines.close[-2])
        print("1 索引：",'datetime',self.data1.lines.datetime.date(1),'close', self.data1.lines.close[1])
        print("2 索引",'datetime', self.data1.lines.datetime.date(2),'close', self.data1.lines.close[2])
        print("从 0 开始往前取3天的收盘价：", self.data1.lines.close.get(ago=0, size=3))
        print("从-1开始往前取3天的收盘价：", self.data1.lines.close.get(ago=-1, size=3))
        print("从-2开始往前取3天的收盘价：", self.data1.lines.close.get(ago=-2, size=3))
        print("line的总长度：", self.data1.buflen())
        
    def next(self):
        print(f"------------- next 的第{self.count+1}次循环 --------------")
        print("当前时点（今日）：",'datetime',self.data1.lines.datetime.date(0),'close', self.data1.lines.close[0])
        print("往前推1天（昨日）：",'datetime',self.data1.lines.datetime.date(-1),'close', self.data1.lines.close[-1])
        print("往前推2天（前日）", 'datetime',self.data1.lines.datetime.date(-2),'close', self.data1.lines.close[-2])
        print("前日、昨日、今日的收盘价：", self.data1.lines.close.get(ago=0, size=3))
        print("往后推1天（明日）：",'datetime',self.data1.lines.datetime.date(1),'close', self.data1.lines.close[1])
        print("往后推2天（明后日）", 'datetime',self.data1.lines.datetime.date(2),'close', self.data1.lines.close[2])
        print("已处理的数据点：", len(self.data1))
        print("line的总长度：", self.data0.buflen())
        self.count += 1
        
cerebro1 = bt.Cerebro()
st_date = datetime(2019,1,2) # 起始时间 2019-01-02
ed_date = datetime(2021,1,28) # 结束时间 2021-01-28
datafeed11 = bt.feeds.PandasData(dataname=data1,fromdate=st_date,todate=ed_date)
cerebro1.adddata(datafeed11, name='600466.SH')
datafeed22 = bt.feeds.PandasData(dataname=data2,fromdate=st_date,todate=ed_date)
cerebro1.adddata(datafeed22, name='603228.SH')
cerebro1.addstrategy(TestStrategy1)
rasult = cerebro1.run()

# ------------- init 中的索引位置-------------
# 0 索引： datetime 2021-01-28 close 54.91980265 
# -1 索引： datetime 2021-01-27 close 55.5952978
# -2 索引 datetime 2021-01-26 close 55.12449815
# 1 索引： datetime 2019-01-02 close 51.12077805 会回到 fromdate
# 2 索引 datetime 2019-01-03 close 50.63976172
# 从 0 开始往前取3天的收盘价： array('d')
# 从-1开始往前取3天的收盘价： array('d', [57.49896595, 55.12449815, 55.5952978])
# 从-2开始往前取3天的收盘价： array('d', [58.1744611, 57.49896595, 55.12449815])
# line的总长度： 506
# ------------- next 的第1次循环 --------------
# 当前时点（今日）： datetime 2019-01-02 close 51.12077805
# 往前推1天（昨日）： datetime 2021-01-28 close 54.91980265
# 往前推2天（前日） datetime 2021-01-27 close 55.5952978
# 前日、昨日、今日的收盘价： array('d')
# 往后推1天（明日）： datetime 2019-01-03 close 50.63976172
# 往后推2天（明后日） datetime 2019-01-04 close 50.4555427
# 已处理的数据点： 1
# line的总长度： 506
# ------------- next 的第2次循环 --------------
# 当前时点（今日）： datetime 2019-01-03 close 50.63976172
# 往前推1天（昨日）： datetime 2019-01-02 close 51.12077805
# 往前推2天（前日） datetime 2021-01-28 close 54.91980265
# 前日、昨日、今日的收盘价： array('d')
# 往后推1天（明日）： datetime 2019-01-04 close 50.4555427
# 往后推2天（明后日） datetime 2019-01-07 close 50.9672622
# 已处理的数据点： 2
# line的总长度： 506
# ------------- next 的第3次循环 --------------
# 当前时点（今日）： datetime 2019-01-04 close 50.4555427
# 往前推1天（昨日）： datetime 2019-01-03 close 50.63976172
# 往前推2天（前日） datetime 2019-01-02 close 51.12077805
# 前日、昨日、今日的收盘价： array('d', [51.12077805, 50.63976172, 50.4555427])
# 往后推1天（明日）： datetime 2019-01-07 close 50.9672622
# 往后推2天（明后日） datetime 2019-01-08 close 50.52718343
# 已处理的数据点： 3
# line的总长度： 506

#### some tips

1. \_\_init__() 中： 
    访问的是整条 line，索引编号也是对整条 line 上所有数据点进行编号的，所以 0 号位置对应导入的行情数据中最晚的那个时间点 2021-01-28，然后依次 backwards；1 号位置对应最早的那个时间点 2019-01-02，然后依次 forwards ；**通过 get() 切片时，如果是从 ago=0 开始取，不会返回数据，从其他索引位置开始取，能返回数据 。**

2. next() 中：  
    a. 由于 next() 是按回测时间点依次循环运行的，所以 next() 中数据点的索引位置是随着回测依次推进而动态变化的：backwards 时对应回测过的、已处理过的那部分 line， forwards 时对应还未回测的那部分 line ；  
    b. 在 next() 中，只要记住 0 是当前回测的时间点（今日），然后站在当前时刻回首过往：-1 是昨日、-2 是前日，依次类推 ；或者站在当前时刻期盼未来：1 是明日、2 是明后日，以此类推 。

3. 获取 line 长度：   
    a. **self.data0.buflen() 返回整条线的总长度，固定不变；**  
    b. 在 next() 中调用 **len(self.data0)，返回的是当前已处理（已回测）的数据长度，会随着回测的推进动态增长。**  

4. datetime 线：  
    a. datetime 线中的时间点存的是数字形式的时间，可以通过 bt.num2date() 方法将其转为“xxxx-xx-xx xx:xx:xx”这种形式；  
    b. 对 datetime 线进行索引时，xxx.date(X) 可以直接以“xxxx-xx-xx xx:xx:xx”的形式返回，X 就是索引位置，可以看做是传统 [X] 索引方式的改进版 。

#### b. bars